# HllSet Relational Algebra

# HllSet 关系代数

**HyperLogLog (HLL)** [2] 算法基于这样一种观察：一个均匀分布的随机数多重集的基数可以通过计算集合中每个数字的二进制表示中尾随零的最大数量来估计。如果观察到的最大尾随零的数量记为 n，那么集合中不同元素的数量估计值为 2^n。[1]

然而，这种估计的方差可能很大。为了解决这个问题，HLL 算法将多重集划分为多个子集。它计算每个子集中数字的尾随零的最大数量，然后使用调和平均将这些估计值结合起来，提供整个集合基数的总体估计。

HLL 数据结构表示为一个 **k 元组 t = (n1, n2, . . . ni, . . . nk)**，其中 **ni** 表示在多重集中为第 **i** 个子集计算的尾随零的最大数量。这个结构允许无损合并两个或多个 HLL，其中合并后的 HLL 相当于对原始数据集的并集进行计算，产生相同的基数估计。

虽然 HLL 结构不支持其他集合操作，例如交集，**但可以通过将元组 t 中的最大零数替换为位向量来增强它**。通过结合 **位向量** 来存储每个子集的所有尾随零数，这种升级后的结构，我们称之为 **HllSets**（HyperLogLog 集合），**可以实现所有集合操作**。

在 Wikipedia 条目 [7] 中，**关系代数** 被描述如下："在数据库理论领域，关系代数是一种理论框架，使用代数结构来建模数据并基于严格定义的语义制定查询。这个理论最初由 Edgar F. Codd 提出。[8]"

在我们的论文中，我们旨在通过 **HllSets** 的视角来探索 **关系代数** 的概念。需要注意的是，这份文件处于初步阶段，预计将进行大量开发。

# 参考文献
1. https://en.wikipedia.org/wiki/HyperLogLog
2. https://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf
3. https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/40671.pdf
4. https://redis.io/docs/data-types/probabilistic/hyperloglogs/
5. https://github.com/ascv/HyperLogLog/blob/master/README.md
6. https://en.wikipedia.org/wiki/Inclusion%E2%80%93exclusion_principle
7. https://en.wikipedia.org/wiki/Algebra_of_sets
8. https://dl.acm.org/doi/10.1145/358396.358400

In [1]:
# using Pkg
# Pkg.activate(".")
# Pkg.instantiate()
# Pkg.add("CSV")
# Pkg.add("Arrow")
# Pkg.add("Tables")
# Pkg.add("JSON3")

## Creating HllSets and applying basic operations

In [2]:
using Random
using FilePathsBase: extension, Path

include("src/sets32.jl")

import .HllSets as set

# Initialize test HllSets
hll1 = set.HllSet{5}(); hll1_seeded = set.HllSet{5}()
hll2 = set.HllSet{5}(); hll2_seeded = set.HllSet{5}()
hll3 = set.HllSet{5}(); hll3_seeded = set.HllSet{5}()
hll4 = set.HllSet{5}(); hll4_seeded = set.HllSet{5}()
hll5 = set.HllSet{5}(); hll5_seeded = set.HllSet{5}()

# Generate datasets from random strings
s1 = Set(randstring(7) for _ in 1:10)
s2 = Set(randstring(7) for _ in 1:15)
s3 = Set(randstring(7) for _ in 1:100)
s4 = Set(randstring(7) for _ in 1:20)
s5 = Set(randstring(7) for _ in 1:130)

# Add datasets to HllSets
set.add!(hll1, s1); set.add!(hll1_seeded, s1, seed=123)
set.add!(hll2, s2); set.add!(hll2_seeded, s2, seed=123)
set.add!(hll3, s3); set.add!(hll3_seeded, s3, seed=123)
set.add!(hll4, s4); set.add!(hll4_seeded, s4, seed=123)
set.add!(hll5, s5); set.add!(hll5_seeded, s5, seed=123)

println(hll1.counts, "\n", count(hll1))
println(hll1_seeded.counts, "\n", count(hll1_seeded))

println("Size of hll1: ", set.sizeof(hll1), "; \nSize of hll1_seeded: ", set.sizeof(hll1_seeded))

UInt32[0x00000001, 0x00000000, 0x00000000, 0x00000000, 0x00000001, 0x00000000, 0x00000002, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000002, 0x00000000, 0x00000008, 0x00000000, 0x00000002, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000002, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000005, 0x00000004, 0x00000000]
11
UInt32[0x00000000, 0x00000000, 0x00000008, 0x00000008, 0x00000000, 0x00000000, 0x00000000, 0x00000001, 0x00000000, 0x00000002, 0x00000001, 0x00000002, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000001, 0x00000002, 0x00000001, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000]
10
Size of hll1: 32; 
Size of hll1_seeded: 32


In [3]:
# Print cardinality of datasets and HllSets side by side
println(length(s1), " : ", count(hll1))
println(length(s2), " : ", count(hll2))
println(length(s3), " : ", count(hll3))
println(length(s4), " : ", count(hll4))
println(length(s5), " : ", count(hll5))

# union
println("\nunion:\n", length(s1 ∪ s2 ∪ s3 ∪ s4 ∪ s5), " : ", count(hll1 ∪ hll2 ∪ hll3 ∪ hll4 ∪ hll5), "\n")

# intersection
println("intersection (standard HllSet with seeded):\n", count(hll1 ∩ hll1_seeded))


10 : 11
15 : 20
100 : 105
20 : 23
130 : 120

union:
275 : 263

intersection (standard HllSet with seeded):
2


### HllSet Universes

In [4]:
A = set.HllSet{5}(); A_123 = set.HllSet{5}()
B = set.HllSet{5}(); B_123 = set.HllSet{5}()
C = set.HllSet{5}(); C_123 = set.HllSet{5}()

items_t1 = Set(["string0", "string1", "string2", "string3", "string4", "string5", "string6", "string7", "string8", "string9", "string10"])
items_t2 = Set(["string3", "string4", "string5", "string6", "string7", "string8", "string9", "string10", "string11"])
items_t3 = Set(["string5", "string6", "string7", "string8", "string9", "string10", "string11"])

set.add!(A, items_t1); set.add!(A_123, items_t1, seed=123)
set.add!(B, items_t2); set.add!(B_123, items_t2, seed=123)
set.add!(C, items_t3); set.add!(C_123, items_t3, seed=123)

In [5]:

# Default and seeded HllSet Universes
U = A ∪ B ∪ C; U_123 = A_123 ∪ B_123 ∪ C_123

# Intersection of 2 Universes is Empty (almost)
println("U ∩ U_123: ", count(U ∩ U_123), "\n")

println("A: ", count(A)); println("A_123: ", count(A_123))
println("B: ", count(B)); println("B_123: ", count(B_123))
println("C: ", count(C)); println("C_123: ", count(C_123))
println("U: ", count(U)); println("U_123: ", count(U_123))

println("AB = A ∩ B: ", count(A ∩ B)); println("AB_123 = A_123 ∩ B_123: ", count(A_123 ∩ B_123))
println("AC = A ∩ C: ", count(A ∩ C)); println("AC_123 = A_123 ∩ C_123: ", count(A_123 ∩ C_123))
println("BC = B ∩ C: ", count(B ∩ C)); println("BC_123 = B_123 ∩ C_123: ", count(B_123 ∩ C_123))

U ∩ U_123: 2

A: 11
A_123: 12
B: 10
B_123: 11
C: 7
C_123: 8
U: 12
U_123: 14
AB = A ∩ B: 9
AB_123 = A_123 ∩ B_123: 10
AC = A ∩ C: 6
AC_123 = A_123 ∩ C_123: 7
BC = B ∩ C: 7
BC_123 = B_123 ∩ C_123: 8


### Defining Probabilities and Conditional Proabilities with HllSets

In [6]:
# Probabilities
println("P(A) = |A| / |U|: ", count(A) / count(U)); println("P(A_123) = |A_123| / |U_123|: ", count(A_123) / count(U_123), "\n")
println("P(B) = |B| / |U|: ", count(B) / count(U)); println("P(B_123) = |B_123| / |U_123|: ", count(B_123) / count(U_123), "\n")
println("P(C) = |C| / |U|: ", count(C) / count(U)); println("P(C_123) = |C_123| / |U_123|: ", count(C_123) / count(U_123), "\n", "\n")

# Conditional Probabilities
println("P(A | B) = |AB| / |B|: ", count(A ∩ B) / count(B)); println("P(A_123 | B_123) = |AB_123| / |B_123|: ", count(A_123 ∩ B_123) / count(B_123), "\n")
println("P(B | A) = |AB| / |A|: ", count(A ∩ B) / count(A)); println("P(A_123 | A_123) = |AB_123| / |A_123|: ", count(A_123 ∩ B_123) / count(A_123), "\n")
println("P(A | C) = |AC| / |C|: ", count(A ∩ C) / count(C)); println("P(A_123 | C_123) = |AC_123| / |C_123|: ", count(A_123 ∩ C_123) / count(C_123), "\n")
println("P(C | A) = |AC| / |A|: ", count(A ∩ C) / count(A)); println("P(A_123 | A_123) = |AC_123| / |A_123|: ", count(A_123 ∩ C_123) / count(A_123), "\n", "\n")

println("P(B | C) = BC / C: ", count(B ∩ C) / count(C))
println("P(C | B) = BC / B: ", count(B ∩ C) / count(B), "\n")

P(A) = |A| / |U|: 0.9166666666666666
P(A_123) = |A_123| / |U_123|: 0.8571428571428571

P(B) = |B| / |U|: 0.8333333333333334
P(B_123) = |B_123| / |U_123|: 0.7857142857142857

P(C) = |C| / |U|: 0.5833333333333334
P(C_123) = |C_123| / |U_123|: 0.5714285714285714


P(A | B) = |AB| / |B|: 0.9
P(A_123 | B_123) = |AB_123| / |B_123|: 0.9090909090909091

P(B | A) = |AB| / |A|: 0.8181818181818182
P(A_123 | A_123) = |AB_123| / |A_123|: 0.8333333333333334

P(A | C) = |AC| / |C|: 0.8571428571428571
P(A_123 | C_123) = |AC_123| / |C_123|: 0.875

P(C | A) = |AC| / |A|: 0.5454545454545454
P(A_123 | A_123) = |AC_123| / |A_123|: 0.5833333333333334


P(B | C) = BC / C: 1.0
P(C | B) = BC / B: 0.7



### **How Many Universes Can We Have?**
### **我们能拥有多少个宇宙？**

这个问题的答案取决于我们使用的操作系统。例如，在 64 位环境中，有 (2^64 - 1)（超过 18 亿亿，或大约 1.8 × 10^19）个可用的不同值，可以用作生成 HllSets 的种子。

如果这些宇宙是由相同的集合构建而成，但使用不同的种子值来为哈希函数生成 HllSets，它们在结构上会非常相似，甚至几乎是相同的。这种现象在两个宇宙中观察到，并且在三个或更多宇宙中也保持一致。

我们将这种现象称为 **HllSets 的纠缠**。

**在 SGS 的框架内，纠缠意味着当相同的数据输入到 HllSets 中时——由不同的哈希函数定义，并可能具有不同的精度参数 (P)——生成的结构往往非常相似，甚至是相同的。**

揭示隐藏结构需要相当大的努力，尤其是在处理非常大的数据集时。**HllSet 纠缠提供了一个机会，可以将一个 SGS 中发现的见解“传送”到另一个已经输入相同或相似数据的 SGS 中**。这种知识的转移可以在不需要移动任何数据或重复发现过程的情况下发生。

### Some other HllSet operations

In [7]:
hll_diff = set.set_xor(A, B)
println("HLL xor: ", count(hll_diff))

hll_int = intersect(A, B)

println("hll_int: ", count(hll_int))

println()
println("=====================================")
hll_comp_1 = set.set_comp(A, B)
println("Comp 1: ", count(hll_comp_1))
println("A: ", count(A))

println()
println("=====================================")
hll_comp_2 = set.set_comp(B, A)
println("Comp 2: ", count(hll_comp_2))
println("B: ", count(B))

HLL xor: 4
hll_int: 9

Comp 1: 3
A: 11

Comp 2: 1
B: 10
